In [1]:
!wget https://github.com/alexey9177950/philosophy/blob/master/ph1.txt?raw=true
!wget https://github.com/alexey9177950/philosophy/blob/master/ph2.txt?raw=true

--2019-02-24 08:24:03--  https://github.com/alexey9177950/philosophy/blob/master/ph1.txt?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/alexey9177950/philosophy/raw/master/ph1.txt [following]
--2019-02-24 08:24:03--  https://github.com/alexey9177950/philosophy/raw/master/ph1.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexey9177950/philosophy/master/ph1.txt [following]
--2019-02-24 08:24:03--  https://raw.githubusercontent.com/alexey9177950/philosophy/master/ph1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting

In [10]:
import os
os.system('mv "ph1.txt?raw=true" ph1.txt')
os.system('mv "ph2.txt?raw=true" ph2.txt')
os.listdir('.')

['.config', 'ph1.txt', 'ph2.txt', 'sample_data']

In [1]:
from collections import Counter
from os.path import join


input_path = "."
txt = open(join(input_path, "ph1.txt"), "r").read() + \
      open(join(input_path, "ph2.txt"), "r").read()
txt = txt.lower().replace('ё', 'е')

cnt = Counter(txt)
charset = [i for i in cnt.keys() if cnt[i] > 1000]
to_char = dict(enumerate(charset))
to_id = dict(zip(charset, range(len(charset))))
charset = set(charset)
data = [to_id[c] for c in txt if c in charset]

# should be ~61m
len(to_id.keys()), len(data)

(102, 61621479)

In [2]:
import keras
import keras.layers as L

BATCH_LEN = 32
N_CHARS = len(to_char)
SEQ_LEN = 100
PER_EPOCH = len(data) // (BATCH_LEN * SEQ_LEN * 30)


def get_model():
    model = keras.models.Sequential()
    model.add(L.InputLayer(input_shape=(100,), batch_size=32, dtype='int32'))
    model.add(L.Embedding(N_CHARS, 100, input_length=SEQ_LEN))
    model.add(L.GRU(1024,
                    recurrent_activation='sigmoid',
                    recurrent_initializer='glorot_uniform',
                    stateful=True,
                    return_sequences=True))
    model.add(L.Dense(N_CHARS))
    model.add(L.Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = get_model()
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (32, 100, 100)            10200     
_________________________________________________________________
gru_1 (GRU)                  (32, 100, 1024)           3456000   
_________________________________________________________________
dense_1 (Dense)              (32, 100, 102)            104550    
_________________________________________________________________
activation_1 (Activation)    (32, 100, 102)            0         
Total params: 3,570,750
Trainable params: 3,570,750
Non-trainable params: 0
_________________________________________________________________


In [3]:
from random import randint
import numpy as np
from keras.utils import to_categorical


def generate_batches():
    X_b = np.zeros((BATCH_LEN, SEQ_LEN))
    y_b = np.zeros((BATCH_LEN, SEQ_LEN, N_CHARS))
    while True:
        for i in range(BATCH_LEN):
            current_idx = randint(0, len(data))
            while current_idx + SEQ_LEN >= len(data):
                current_idx = randint(0, len(data))
            
            X_b[i, :] = data[current_idx : current_idx + SEQ_LEN]
            temp_y = data[current_idx + 1: current_idx + SEQ_LEN + 1]
            y_b[i, :, :] = to_categorical(temp_y, num_classes=N_CHARS)
        yield X_b, y_b


num_epochs = 8
history = model.fit_generator(generate_batches(),
                    PER_EPOCH,
                    num_epochs)
model.save('ph_model.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/8
641/641 [==============================] - 191s 298ms/step - loss: 2.6972
Epoch 2/8
641/641 [==============================] - 190s 296ms/step - loss: 2.1841
Epoch 3/8
641/641 [==============================] - 189s 295ms/step - loss: 1.8864
Epoch 4/8
641/641 [==============================] - 189s 295ms/step - loss: 1.8451
Epoch 5/8
641/641 [==============================] - 189s 294ms/step - loss: 1.8427
Epoch 6/8
641/641 [==============================] - 187s 292ms/step - loss: 1.6668
Epoch 7/8
641/641 [==============================] - 188s 293ms/step - loss: 1.6663
Epoch 8/8
641/641 [==============================] - 187s 292ms/step - loss: 1.6819


In [0]:
from importlib import reload
import google
reload(google)
from google.colab.files import download
download('ph_model.h5')

In [0]:
def decode(seq):
    return "".join(to_char[round(i)] for i in seq)
  
def encode(seq):
    return [to_id[i] for i in seq]

  
def gen_txt(prefix="", power=2):
    model.reset_states()
    X_b, y_b = next(iter(generate_batches()))
    cur_vec = X_b[0]
    cur_vec[-len(prefix):] = np.array(encode(prefix))
    cur_vec[:-len(prefix)] = np.array(encode(" " * (100 - len(prefix))))
    cur_vec = np.array([cur_vec for i in range(32)])

    texts = [prefix for i in range(32)]

    for i in range(900):
      output = model.predict(cur_vec)[:,-1]
      output = (output ** power) / (output ** power).sum(axis=1).reshape(32, 1)
      for j in range(32):
          new_val = np.random.choice(range(N_CHARS), p=output[j])
          cur_vec[j] = np.concatenate((cur_vec[j][1:], [new_val]))
          texts[j] += decode(cur_vec[j][-1:])
    return texts


def print_txt(texts):
    for i in range(len(texts)):
      print("NUMBER", i + 1)
      for j, c in enumerate(texts[i]):
        print(c, end="")
        if j % 80 == 79:
          print()
      print("\n\n")

In [5]:
%%time

TXT_NUM = 5

print_txt(gen_txt(" ", 1.5)[:TXT_NUM])

NUMBER 1
 применяет за кородкий его собственное начало об иного предмета. таких объективн
ых суждений от арховом раньше универсального и прекрасного трансцендентального с
пособность она уступает государство более умер в том восприятии, которое есть см
ысла и даже различение полной оставляющей всех двух состояний: меня не указывал,
 что дело утверждать, что это простое происходит от себя и объясняется то время 
как простое, но не развернуть средние истинные структуры и других. но оно стало 
быть, мы не взнысили конституцию в последнем сознании не прекрасив его для них п
ринцип. но все задачс возможностямистической деятельности не остает проблемы суб
ъективной план. а, искал предметов представления к представлению, которая не дет
с красоторый выступает и, судебное понятие в зависит от старого существа, однако
 же другое, не подрасаалистичники философии, то считаюоченные последних случайны
е сознания здравания 


NUMBER 2
 которых до сих пор прочивает все свои молилы, однако, согласиться 

In [6]:
print_txt(gen_txt(" ", 2)[:TXT_NUM])

NUMBER 1
 протяженности и подобно тому, что из них относится к проблематическую философск
ую количествую метафизику истории и весьма совершенно отделяет собственность, по
 своему порогу совершенно совершенно произойдет в посредством различия и чистого
 разума.
поэтому они представляют собой в обществе и платоновского порядка. посл
едняя требуется также определяло бы объективное, составляющие он проду бытиями (
такого рода их совсем не столько совсем не не находится в свое основание от прос
того разума, в качестве понятия общественности в собой  как раз он даны в опыте,
 в каком-то смысле есть противоположное пределение предметов и простое простое с
остояние и мы предполагаем?
что касается социального предмета, направленное на в
оздух духовного пространства. но поскольку он представляет собой некоторые и под
обно свободными и превратили в речи и совершенно связывается с помощью которого 
движения в нем воспри


NUMBER 2
 свободное осуществление (или противоположность и трудно с помощью 

In [7]:
print_txt(gen_txt(" ", 3)[:TXT_NUM])

NUMBER 1
 как свое положение в полной теории и не просто производства и просто не обладае
т не представляется объективной свободы, которая определяет собственность подобн
ого предмета, постоянно подразумевающее в состоянии сознания просто последнего п
редмета, то в отношении к своему созерцанию он предполагает возможность познания
 и символического противоречия сознания в своем собственном смысле и принимает с
вою проблему, которая в своем отношении к своему объекту и причину возникает и о
т него настоящим образом в состоянии производства с помощью которой он становитс
я понятием и определяющее понятие всем и тождественно сознанием своего предмета,
 который оно совершается в своем действии. но в том случае, если он это на основ
ании образованной страницей и представляется прекрасной и не признает никакого о
тношения к собственному предмету, то есть подлинное представление о простой рефл
ексии постоянно поста


NUMBER 2
 под вопросом от предмета, есть простое понятие предметов поверхнос

In [8]:
print_txt(gen_txt(" ", 5)[:TXT_NUM])

NUMBER 1
 различие между собой, совершенно от своего происхождения и представляет собой п
оследняя противополагается другой стороны, то есть постоянно противополагает сво
е производство в том, что он выступал свое предмет и постоянно по своему существ
у, не может быть просто от них предметов и в своем собственном смысле. но все эт
о понятие не может быть принимаемым в своем собственном смысле. но в простой мат
ерии просто понимается о себе и для себя. таким образом, он открывает собственно
сть постоянно просто постоянно поставить в состоянии понятий и предметов, поскол
ьку он представляет собой постоянно постоянно совершенно от него самого себя, и 
в отношении других предметов, которые представляют собою совершенно отрицание ст
арого предмета, то есть другого понятия и противоположности и поставить в себе и
 для себя. но оно представляется не просто не познает себя и постоянно понимают 
свою полноту воли и п


NUMBER 2
 собственной степени просто противополагает простое предметом социа

In [10]:
%%time

print_txt(gen_txt(" движение исчезновения бытия ничто ", 1.75)[:TXT_NUM])

NUMBER 1
 движение исчезновения бытия ничто применение слово «целому подчинение труда» по
том и никакого русского страна в своем отношении, в котором он сам постоянно обо
гнавляется сверху и в том или ином смысле, которое он хочет зависит то же время 
другие метафизики и не может быть пониманием о привкости приняты вся в первом сл
учае, мы смешиваем тогда как последние присутствуют к чувству проблематического 
события. поэтому для каждого общего понятия не пребывает в своем постепенном смы
сле и ее проблематического конца: а в душе полностью безумия может быть признаем
ое наивной идеальности, то с другой стороны, разнообразировать за себя и социаль
ных процессов. он был противоречит на предметов, оторванных и которые словно не 
произой волной и объективности, центротической склонности к этому естественной п
олнотой свободы и образуют себя антионотомской политикой в управлении свойства и
 человека, то всякого опыта, а то и не может быть также


NUMBER 2
 движение исчезновения бытия ничт

In [11]:
%%time
print_txt(gen_txt(" ничто перешло в бытие ", 2.25)[:TXT_NUM])

NUMBER 1
 ничто перешло в бытие и субстанциальное политический закон от себя в своем дейс
твии, но в отношении к будущему и постоянно принять в состоянии действительности
 противоположности социальной потребности в качестве своего разума, и от которог
о я оставляю за последний смысл и от «собственной свободы». в своем поступке мож
ет быть понимание до сих пор открывается в самом деле, он находится в последнем 
рассмотрении он ставил в себе самом деле, по нет полного производства и стоит вы
сказать на себя самого себя, не просто не знает и во всем том, что он представля
ется, поскольку оно позитивно высшее собственное содержание о боге и объясняется
 не потому, что его существуют и общественно и все тем более просто не потреблен
ием по природе остается объективной и необходимостью полностью производства и пр
отивоположности и высшей степени и в своем роде, против своего общества не обосн
овала и потому что каждое из них понятие о 


NUMBER 2
 ничто перешло в бытие вещей и предметом стат

In [12]:
%%time
print_txt(gen_txt(" смысл жизни в том, что ", 2.25)[:TXT_NUM])

NUMBER 1
 смысл жизни в том, что в своем собственном смысле следует понимать в качестве п
роизводства и противополагается на этой почве и полностью субъективности; но пос
ледняя уже не была связана с самого себя. но просто от них люди не понимается в 
его объективной проблемы в мире, на которой то на самом деле они должны принимат
ь себя существование противополагается противоположностью к тому, что на противо
полагает собой в том, что они поступает на том науке и которых есть тот же порог
, которая он не поступает на последние стоит в том, что он обладает собственност
и не подвергается человека, с другой стороны, и вы обосновании и необходимости и
 в особенности социальной проблемы сообщения к третьему собственному пространств
у, в своем разуме, однако, поскольку оно продолжает не простое понятие о субъект
е и страдание и представляется чем-то состоянием и высказыванием и представляетс
я собственной стороны, на которой он ставит 


NUMBER 2
 смысл жизни в том, что проблематическая про

In [13]:
print_txt(gen_txt(" наука доказала, что", 1.75)[:TXT_NUM])

NUMBER 1
 наука доказала, что он соответствует собственной своей собственной смертной про
тивоположность с величины на обеспечивания несколько всех видимости, которая при
нимается в своем содержании прекрасного порядка, но не знаю религиозной природы 
его. проблема не может быть полезно предпосылаться в упорном и ластическому исто
рию социального предмета. то, что можно сделать собой долга, они представляют се
бе высшей собственной величины, в которой оно не оставляет собственное пребывани
е в собственном смысле. может обстоит дело и способность во всяком случае и что-
то ни одно и то же самое в нем нелепо произойдет большее состояние любовь к друг
ому. у него не может быть представляемо, и тот же простое и для себя существован
ие момента. но они должны подвергнуть настоящим преимуществом и мало что преимущ
ественно и не представляет собою духовное состояние постигнет на себя и для нас 
от них не обстоит дело даже противоречит


NUMBER 2
 наука доказала, что начинается не только одно в

In [14]:
print_txt(gen_txt(" мировое правительство замышляет ", 1.75)[:TXT_NUM])

NUMBER 1
 мировое правительство замышляет свое производство от действительности и всех по
добной любви, самой своей почвы не может быть поэтому в сознании не только на др
угое в мире, которое все постигается в моих настоящей природы и постоянно против
орвсенной последовательности чистого разума. любопническая философия обладает не
 обезумностью, а не не по себе, имеет без свои недостаточно меня в отношении к о
бъекте, а не производит противоположности собственной политики, т.е. материально
сть полнота опыта, то он не оставляет они сохраняют нам, что просто смысл рассмо
трения вообще в своем значении не обязательно получает просто не высказывание и 
они контракт и предразумений и смерти и противополагает некоторое государство, н
о он ваш оставляет торжество социального бытия и все равно в состоянии подменяет
ся связь занимать внушение к этому систему его субъекта, то ге существует не о т
ом, что различие субъективности понимания духа по сво


NUMBER 2
 мировое правительство замышляет в 